# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

In [1208]:
import pandas as pd
import numpy as np
import re

In [1209]:
nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


In [1210]:
#cities.head(10)

In [1211]:
#nhl_df

In [1212]:
nhldat = nhl_df[nhl_df['year'] == 2018]
#nhldat.head()

In [1213]:
cities.columns

Index(['Metropolitan area', 'Population (2016 est.)[8]', 'NFL', 'MLB', 'NBA',
       'NHL'],
      dtype='object')

## Cleaning 2 Data

### Wikipedia (metropolitan cities - citiesnhl)

In [1214]:
citiesnhl = cities.copy()
citiesnhl.drop(citiesnhl.columns[2:5], axis=1, inplace=True)
citiesnhl.rename({'Population (2016 est.)[8]': 'Population'}, axis=1, inplace=True)

In [1215]:
#citiesnhl.head() 

In [1216]:
citiesnhl['NHL'] = citiesnhl['NHL'].apply(lambda x: re.sub(r'\[[\w ]*\]','', x)) 
citiesnhl['NHL'] = citiesnhl['NHL'].apply(lambda x: re.sub(r'\—','', x)) 
#citiesnhl['NHL'] = citiesnhl['NHL'].apply(lambda x: re.sub('-','', x)) 

citiesnhl[citiesnhl['NHL'] == '—'] = np.nan

In [1217]:
#replace empty values with nan
nans = citiesnhl[citiesnhl['NHL'] == ''].index
citiesnhl['NHL'].iloc[nans] = np.nan

### NHL dadta (nhldat)

In [1218]:
##Remove Division from rows and create a 'Division' that specifies each team's division
nhldat2 = nhldat.copy()
nhldat2['Division'] = 0

In [1219]:

nhldat2['Division'] = 0
idx = 0
name='0'
for i,tm in enumerate(nhldat2['team']):
    name = tm
    if 'Division' in tm:
        nhldat2['Division'].iloc[idx+1:i] = tm
        idx = i
        #nhldat2.drop(i, axis=0, inplace=True)
        
nhldat2['Division'].iloc[idx+1:] = name       



In [1220]:
tp = nhldat2[nhldat2['Division'] == 0]

nhldat3=nhldat2[~nhldat2.index.isin(tp.index)]
#nhldat3.head()

In [1221]:
x = nhldat3['team'].iloc[0]
x.strip('*')

'Tampa Bay Lightning'

In [1222]:
##remove asteriks
nhldat3['team'] = nhldat3['team'].apply(lambda x: x.strip('*')) 

### Two Data

In [1223]:
nhldat3

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,Division
1,Tampa Bay Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL,Metropolitan Division
2,Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,Metropolitan Division
3,Toronto Maple Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL,Metropolitan Division
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL,Metropolitan Division
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL,Metropolitan Division
6,Montreal Canadiens,82,29,40,13,71,.433,209,264,-0.68,0.00,.378,27,2018,NHL,Metropolitan Division
7,Ottawa Senators,82,28,43,11,67,.409,221,291,-0.85,0.00,.372,26,2018,NHL,Metropolitan Division
8,Buffalo Sabres,82,25,45,12,62,.378,199,280,-0.98,0.01,.311,24,2018,NHL,Metropolitan Division
10,Washington Capitals,82,49,26,7,105,.640,259,239,0.21,-0.04,.585,46,2018,NHL,Central Division
11,Pittsburgh Penguins,82,47,29,6,100,.610,272,250,0.23,-0.04,.573,45,2018,NHL,Central Division


In [1224]:
citiesnhl

,Metropolitan area,Population,NHL
0,New York City,20153634,RangersIslandersDevils
1,Los Angeles,13310447,KingsDucks
2,San Francisco Bay Area,6657982,Sharks
3,Chicago,9512999,Blackhawks
4,Dallas–Fort Worth,7233323,Stars
5,"Washington, D.C.",6131977,Capitals
6,Philadelphia,6070500,Flyers
7,Boston,4794447,Bruins
8,Minneapolis–Saint Paul,3551036,Wild
9,Denver,2853077,Avalanche


### Find each NHL team's city in citiesnhl 

In [1225]:
set1 = list(nhldat3['team'].dropna())
set2 = list(citiesnhl['Metropolitan area'].dropna())
print(set1,'\n\n')
print(set2)

['Tampa Bay Lightning', 'Boston Bruins', 'Toronto Maple Leafs', 'Florida Panthers', 'Detroit Red Wings', 'Montreal Canadiens', 'Ottawa Senators', 'Buffalo Sabres', 'Washington Capitals', 'Pittsburgh Penguins', 'Philadelphia Flyers', 'Columbus Blue Jackets', 'New Jersey Devils', 'Carolina Hurricanes', 'New York Islanders', 'New York Rangers', 'Nashville Predators', 'Winnipeg Jets', 'Minnesota Wild', 'Colorado Avalanche', 'St. Louis Blues', 'Dallas Stars', 'Chicago Blackhawks', 'Vegas Golden Knights', 'Anaheim Ducks', 'San Jose Sharks', 'Los Angeles Kings', 'Calgary Flames', 'Edmonton Oilers', 'Vancouver Canucks', 'Arizona Coyotes'] 


['New York City', 'Los Angeles', 'San Francisco Bay Area', 'Chicago', 'Dallas–Fort Worth', 'Washington, D.C.', 'Philadelphia', 'Boston', 'Minneapolis–Saint Paul', 'Denver', 'Miami–Fort Lauderdale', 'Phoenix', 'Detroit', 'Toronto', 'Houston', 'Atlanta', 'Tampa Bay Area', 'Pittsburgh', 'Cleveland', 'Seattle', 'Cincinnati', 'Kansas City', 'St. Louis', 'Baltim

In [1226]:
#x = 'Tampa Bay Lightning'
#L = list(x.split(' '))
Dik={}
for s1 in set1:
    L = list(s1.split(' '))
    for s2 in set2:   
        if any(l in s2 for l in L):
            Dik[s1] = s2
            
#any(l in st for l in L for st in set2)            

In [1227]:
Dik

{'Tampa Bay Lightning': 'Green Bay',
 'Boston Bruins': 'Boston',
 'Toronto Maple Leafs': 'Toronto',
 'Detroit Red Wings': 'Detroit',
 'Montreal Canadiens': 'Montreal',
 'Ottawa Senators': 'Ottawa',
 'Buffalo Sabres': 'Buffalo',
 'Washington Capitals': 'Washington, D.C.',
 'Pittsburgh Penguins': 'Pittsburgh',
 'Philadelphia Flyers': 'Philadelphia',
 'Columbus Blue Jackets': 'Columbus',
 'New Jersey Devils': 'New Orleans',
 'New York Islanders': 'New Orleans',
 'New York Rangers': 'New Orleans',
 'Nashville Predators': 'Nashville',
 'Winnipeg Jets': 'Winnipeg',
 'St. Louis Blues': 'St. Louis',
 'Dallas Stars': 'Dallas–Fort Worth',
 'Chicago Blackhawks': 'Chicago',
 'Vegas Golden Knights': 'Las Vegas',
 'San Jose Sharks': 'San Antonio',
 'Los Angeles Kings': 'Los Angeles',
 'Calgary Flames': 'Calgary',
 'Edmonton Oilers': 'Edmonton',
 'Vancouver Canucks': 'Vancouver'}

In [1228]:
rems = set(set1) - set(Dik.keys())
rems = list(rems)
rems

['Colorado Avalanche',
 'Arizona Coyotes',
 'Florida Panthers',
 'Carolina Hurricanes',
 'Anaheim Ducks',
 'Minnesota Wild']

In [1229]:
Dik[rems[1]] = 'Phoenix'
Dik[rems[3]] = 'Raleigh'
Dik[rems[0]] = 'Denver'
Dik[rems[2]] = 'Miami–Fort'
Dik[rems[5]] = 'Minneapolis–Saint Paul'


In [1230]:
Dik

{'Tampa Bay Lightning': 'Green Bay',
 'Boston Bruins': 'Boston',
 'Toronto Maple Leafs': 'Toronto',
 'Detroit Red Wings': 'Detroit',
 'Montreal Canadiens': 'Montreal',
 'Ottawa Senators': 'Ottawa',
 'Buffalo Sabres': 'Buffalo',
 'Washington Capitals': 'Washington, D.C.',
 'Pittsburgh Penguins': 'Pittsburgh',
 'Philadelphia Flyers': 'Philadelphia',
 'Columbus Blue Jackets': 'Columbus',
 'New Jersey Devils': 'New Orleans',
 'New York Islanders': 'New Orleans',
 'New York Rangers': 'New Orleans',
 'Nashville Predators': 'Nashville',
 'Winnipeg Jets': 'Winnipeg',
 'St. Louis Blues': 'St. Louis',
 'Dallas Stars': 'Dallas–Fort Worth',
 'Chicago Blackhawks': 'Chicago',
 'Vegas Golden Knights': 'Las Vegas',
 'San Jose Sharks': 'San Antonio',
 'Los Angeles Kings': 'Los Angeles',
 'Calgary Flames': 'Calgary',
 'Edmonton Oilers': 'Edmonton',
 'Vancouver Canucks': 'Vancouver',
 'Arizona Coyotes': 'Phoenix',
 'Carolina Hurricanes': 'Raleigh',
 'Colorado Avalanche': 'Denver',
 'Florida Panthers': 'M

#x = 'Tampa Bay Lightning'
#L = list(x.split(' '))
Dik={}
for s1 in set1:
    L = s1.split(' ')
    I = [s2.find(l) for l in L for s2 in set2]
    if len(set(I)) != 1:
        Dik[s1] = s2
        
#any(l in st for l in L for st in set2)            

In [1231]:
nhldat3['team'].iloc[0] in Dik

True

In [1232]:
nhldat4 = nhldat3.copy()
def findcity(x):
    if x in Dik:
        return Dik[x]
    else:
        return np.nan
nhldat4['city'] = nhldat4['team'].apply(findcity)

In [1233]:
cols = nhldat4.columns.tolist()
#cols = cols[-1:] + cols[:-1]
#nhldat4.columns = cols
#nhldat4
cols
cols.insert(1, cols[-1])
cols = cols[:-1]
nhldat4 = nhldat4[cols]

In [1234]:
nhldat4.set_index('team', inplace=True)
nhldat4

,city,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,Division
team,,,,,,,,,,,,,,,,
Tampa Bay Lightning,Green Bay,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL,Metropolitan Division
Boston Bruins,Boston,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,Metropolitan Division
Toronto Maple Leafs,Toronto,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL,Metropolitan Division
Florida Panthers,Miami–Fort,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL,Metropolitan Division
Detroit Red Wings,Detroit,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL,Metropolitan Division
Montreal Canadiens,Montreal,82,29,40,13,71,.433,209,264,-0.68,0.00,.378,27,2018,NHL,Metropolitan Division
Ottawa Senators,Ottawa,82,28,43,11,67,.409,221,291,-0.85,0.00,.372,26,2018,NHL,Metropolitan Division
Buffalo Sabres,Buffalo,82,25,45,12,62,.378,199,280,-0.98,0.01,.311,24,2018,NHL,Metropolitan Division
Washington Capitals,"Washington, D.C.",82,49,26,7,105,.640,259,239,0.21,-0.04,.585,46,2018,NHL,Central Division


#bring others columns of citynhl to nhldat4
Pop=[]
NHL=[]
for i,c in enumerate(nhldat4['city']):
    if c in list(citiesnhl['Metropolitan area']):
        Pop.append(citiesnhl.iloc[i][1])
        NHL.append(citiesnhl.iloc[i][2])
    else:
        Pop.append(np.nan)
        NHL.append(np.nan)
        
nhldat4['Pop'] = Pop
nhldat4['NHL'] = NHL

In [1237]:
#bring others columns of citynhl to nhldat4

nhldat4['Pop'] = 0
nhldat4['NHL'] = 0
for i,c in enumerate(list(nhldat4['city'].dropna())):
    if c in list(citiesnhl['Metropolitan area'].dropna()):
            idx = nhldat4[nhldat4['city'] == c].index
            #nhldat4['Pop'].loc[idx] = citiesnhl.iloc[i][1]
            #nhldat4['NHL'].loc[idx] = citiesnhl.iloc[i][2] 
            pop = citiesnhl[citiesnhl['Metropolitan area'] == c].values[0][1]
            NHL = citiesnhl[citiesnhl['Metropolitan area'] == c].values[0][2]
            nhldat4['Pop'].loc[idx] = pop
            nhldat4['NHL'].loc[idx] = NHL

#bring others columns of citynhl to nhldat4

nhldat4['Pop'] = 0
nhldat4['NHL'] = 0
for i,c in enumerate(list(nhldat4['city'].dropna())):
    if c in list(citiesnhl['Metropolitan area'].dropna()):
            idx = nhldat4[nhldat4['city'] == c].index
            #nhldat4['Pop'].loc[idx] = citiesnhl.iloc[i][1]
            #nhldat4['NHL'].loc[idx] = citiesnhl.iloc[i][2] 
            pop = citiesnhl[citiesnhl['Metropolitan area'] == c]['Population']
            NHL = citiesnhl[citiesnhl['Metropolitan area'] == c]['NHL']
            #nhldat4['Pop'].loc[idx] = 3
            #nhldat4['NHL'].loc[idx] = 4
            print(type(pop), '\n\n', type(NHL))
        


In [1194]:
c = list(nhldat4['city'].dropna())[0]

L = list(citiesnhl['Metropolitan area'].dropna())
print(c,'\n', L)

Green Bay 
 ['New York City', 'Los Angeles', 'San Francisco Bay Area', 'Chicago', 'Dallas–Fort Worth', 'Washington, D.C.', 'Philadelphia', 'Boston', 'Minneapolis–Saint Paul', 'Denver', 'Miami–Fort Lauderdale', 'Phoenix', 'Detroit', 'Toronto', 'Houston', 'Atlanta', 'Tampa Bay Area', 'Pittsburgh', 'Cleveland', 'Seattle', 'Cincinnati', 'Kansas City', 'St. Louis', 'Baltimore', 'Charlotte', 'Indianapolis', 'Nashville', 'Milwaukee', 'New Orleans', 'Buffalo', 'Montreal', 'Vancouver', 'Orlando', 'Portland', 'Columbus', 'Calgary', 'Ottawa', 'Edmonton', 'Salt Lake City', 'Winnipeg', 'San Diego', 'San Antonio', 'Sacramento', 'Las Vegas', 'Jacksonville', 'Oklahoma City', 'Memphis', 'Raleigh', 'Green Bay', 'Hamilton', 'Regina']


In [1195]:
c in L

True

In [1196]:
idx = nhldat4[nhldat4['city'] == c]
idx

,city,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,Division,Pop,NHL
team,,,,,,,,,,,,,,,,,,
Tampa Bay Lightning,Green Bay,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL,Metropolitan Division,0,0


nan

In [1238]:
nhldat4

,city,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,Division,Pop,NHL
team,,,,,,,,,,,,,,,,,,
Tampa Bay Lightning,Green Bay,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL,Metropolitan Division,318236,NaN
Boston Bruins,Boston,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,Metropolitan Division,4794447,Bruins
Toronto Maple Leafs,Toronto,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL,Metropolitan Division,5928040,Maple Leafs
Florida Panthers,Miami–Fort,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL,Metropolitan Division,0,0
Detroit Red Wings,Detroit,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL,Metropolitan Division,4297617,Red Wings
Montreal Canadiens,Montreal,82,29,40,13,71,.433,209,264,-0.68,0.00,.378,27,2018,NHL,Metropolitan Division,4098927,Canadiens
Ottawa Senators,Ottawa,82,28,43,11,67,.409,221,291,-0.85,0.00,.372,26,2018,NHL,Metropolitan Division,1323783,Senators
Buffalo Sabres,Buffalo,82,25,45,12,62,.378,199,280,-0.98,0.01,.311,24,2018,NHL,Metropolitan Division,1132804,Sabres
Washington Capitals,"Washington, D.C.",82,49,26,7,105,.640,259,239,0.21,-0.04,.585,46,2018,NHL,Central Division,6131977,Capitals


In [1239]:
citiesnhl.sort_values('Metropolitan area')

,Metropolitan area,Population,NHL
15,Atlanta,5789700,NaN
23,Baltimore,2798886,NaN
7,Boston,4794447,Bruins
29,Buffalo,1132804,Sabres
35,Calgary,1392609,Flames
24,Charlotte,2474314,NaN
3,Chicago,9512999,Blackhawks
20,Cincinnati,2165139,NaN
18,Cleveland,2055612,NaN
34,Columbus,2041520,Blue Jackets


In [559]:
citiesnhl2['Metropolitan area'].isin(nhldat4['city'])

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10    False
11     True
12     True
13     True
16     True
17     True
22     True
26     True
29     True
30     True
31     True
34     True
35     True
36     True
37     True
39     True
43     True
47     True
Name: Metropolitan area, dtype: bool

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values